In [5]:
import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

In [20]:
apple = yf.Ticker("TSLA")
stock_data = apple.history(period="1y", interval="1d")
stock_data.index = stock_data.index.tz_localize(None)
stock_data["SMA 10"] = stock_data["Close"].rolling(window=10).mean()
stock_data["SMA 50"] = stock_data["Close"].rolling(window=50).mean()
stock_data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,SMA 10,SMA 50
Date,,,,,,,,,
2024-07-19,247.789993,249.440002,236.830002,239.199997,87403900,0.0,0.0,NaN,NaN
2024-07-22,244.210007,253.210007,243.750000,251.509995,101225400,0.0,0.0,NaN,NaN
2024-07-23,253.600006,255.759995,245.630005,246.380005,111928200,0.0,0.0,NaN,NaN
2024-07-24,225.419998,225.990005,214.710007,215.990005,167942900,0.0,0.0,NaN,NaN
2024-07-25,216.800003,226.000000,216.229996,220.250000,100636500,0.0,0.0,NaN,NaN


In [21]:
fig = go.Figure(data=[go.Candlestick(
    x=stock_data.index,
    open=stock_data.Open,
    close=stock_data.Close,
    high=stock_data.High,
    low=stock_data.Low
)])

fig.add_trace(go.Scatter(
    x=stock_data.index,
    y=stock_data["SMA 10"],
    mode='lines',
    name='SMA 10',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=stock_data.index,
    y=stock_data["SMA 50"],
    mode='lines',
    name='SMA 50',
    line=dict(color='orange')
))


fig.show()

In [22]:
# finding where the market is trending up or down

# define derivative
def differentiate(current_value, previous_value, interval):
    return (current_value - previous_value) / interval

def subtract(time1, time2):
    return (time1.timestamp() - time2.timestamp())

vals10 = []
vals50 = []

for i in range(len(stock_data["SMA 10"])):
    if i == 0:
        val = np.nan
    else:
        val = differentiate(stock_data["SMA 10"].iloc[i], stock_data["SMA 10"].iloc[i-1], subtract(stock_data.index[i], stock_data.index[i-1]))
    vals10.append(val)

for i in range(len(stock_data["SMA 50"])):
    if i == 0:
        val = np.nan
    else:
        val = differentiate(stock_data["SMA 50"].iloc[i], stock_data["SMA 50"].iloc[i-1], subtract(stock_data.index[i], stock_data.index[i-1]))
    vals50.append(val)
    

stock_data["SMA 10 D"] = vals10
stock_data["SMA 50 D"] = vals50

In [23]:
increasing10 = []
for i in range(len(stock_data.index)):
    if stock_data["SMA 10 D"].iloc[i] >= 0:
        increasing10.append(True)
    elif stock_data["SMA 10 D"].iloc[i] < 0:
        increasing10.append(False)
increasing50 = []
for i in range(len(stock_data.index)):
    if stock_data["SMA 50 D"].iloc[i] >= 0:
        increasing50.append(True)
    elif stock_data["SMA 50 D"].iloc[i] < 0:
        increasing50.append(False)

In [24]:
from itertools import groupby

def find_bool_blocks(lst):
    blocks = []
    idx = 0
    for val, group in groupby(lst):
        group_len = len(list(group))
        blocks.append((idx, idx + group_len - 1, val))
        idx += group_len
    return blocks

for val in find_bool_blocks(increasing10):
    if val[2]:
        fig.add_vrect(x0=stock_data.index[val[0]], x1=stock_data.index[val[1]], opacity=0.3, fillcolor="green")
    else:
        fig.add_vrect(x0=stock_data.index[val[0]], x1=stock_data.index[val[1]], opacity=0.3, fillcolor="red")
fig.show()

In [25]:
# trading strategy, if short term increases and long term increases then i will buy 2 - bad market is sell 1
money = 10_000
portfolio = {
    "Apple": 0
}

value_of_portfolio = [np.nan] * len(stock_data)

def transact(mode: str, portfolio: dict, ticker: str, money: float, shares: int, price: float):
    if mode.lower() == "buy":
        portfolio[ticker] += shares
        money -= price * shares
    elif mode.lower() == "sell":
        portfolio[ticker] -= shares
        money += price * shares
    else:
        raise ValueError("Mode is not buy or sell")
    return money

for i in range(len(stock_data.index)):
    price = stock_data.Close.iloc[i]
    if i <= 50 and i % 10 == 0:
        money = transact(mode="buy", portfolio=portfolio, ticker="Apple", money=money, shares=2, price=price)
        pass
    if stock_data["SMA 50 D"].iloc[i] > 0 and (5*price < money):
        if stock_data["SMA 10 D"].iloc[i] > 0:
            money = transact(mode="buy", portfolio=portfolio, ticker="Apple", money=money, shares=5, price=price)
            print(f"Bought! Price of Share {price}, Shares: {portfolio['Apple']}, Money = {money}")
        else:
            portfolio["Apple"] += 0
    
    if stock_data["SMA 50 D"].iloc[i] <= 0:
        if stock_data["SMA 10 D"].iloc[i] > 0:
            portfolio["Apple"] += 0
        else:
            if portfolio["Apple"] >= 5:
                money = transact(mode="sell", portfolio=portfolio, ticker="Apple", money=money, shares=5, price=price)
                print(f"Sold! Shares: {portfolio['Apple']}, Money = {money}")
            else:
                portfolio["Apple"] -= 0
    value_of_portfolio[i] = (portfolio["Apple"] * price + money)

print(value_of_portfolio)

Bought! Price of Share 261.6300048828125, Shares: 17, Money = 5960.829986572266
Bought! Price of Share 258.0199890136719, Shares: 22, Money = 4670.730041503906
Bought! Price of Share 249.02000427246094, Shares: 27, Money = 3425.6300201416016
Bought! Price of Share 250.0800018310547, Shares: 32, Money = 2175.230010986328
Bought! Price of Share 260.4800109863281, Shares: 37, Money = 872.8299560546875
Sold! Shares: 32, Money = 2515.3299560546875
Sold! Shares: 27, Money = 4294.529937744141
Sold! Shares: 22, Money = 6066.5299072265625
Sold! Shares: 17, Money = 7755.529846191406
Sold! Shares: 12, Money = 9408.17984008789
Sold! Shares: 7, Money = 10922.179779052734
Sold! Shares: 2, Money = 12376.179718017578
Bought! Price of Share 298.260009765625, Shares: 7, Money = 10884.879669189453
Bought! Price of Share 318.3800048828125, Shares: 12, Money = 9292.97964477539
Bought! Price of Share 334.07000732421875, Shares: 17, Money = 7622.629608154297
Bought! Price of Share 347.67999267578125, Shares:

In [26]:
money = 10_000
portfolio = {
    "Apple": 0
}

max_num_of_shares = money // stock_data.Open.iloc[0]

portfolio["Apple"] = max_num_of_shares
money -= max_num_of_shares * stock_data.Open.iloc[0]

baseline = []
for i in range(len(stock_data.index)):
    baseline.append(stock_data.Close.iloc[i] * portfolio["Apple"] + money)

In [ ]:
portfolio_fig = go.Figure(
    data=[
        go.Scatter(
            x = stock_data.index,
            y = baseline
        )
    ]
)

portfolio_fig.add_trace(
    go.Scatter(
        x = stock_data.index,
        y = value_of_portfolio
    )
)

portfolio_fig.show()

In [28]:
# creating Portfolio class

class Portfolio:
    def __init__(self, initial_money: float, share_holdings: dict):
        self.money = initial_money
        self.portfolio = share_holdings
    def sell(self, ticker: str, shares: int, price: float):
        if self.portfolio[ticker] >= shares and shares != 0:
            self.portfolio[ticker] -= shares
            self.money += price * shares
    def buy(self, ticker: str, shares: int, price: float):
        if price * shares <= self.money and shares != 0:
            self.portfolio[ticker] += shares
            self.money -= price * shares
    def add(self, ticker: str, shares: int):
        self.portfolio[ticker] += shares
    def subtract(self, ticker: str, shares: int):
        self.portfolio[ticker] -= shares
    def get_value(self, ticker, price):
        return self.portfolio[ticker] * price + self.money

In [29]:
portfolio = Portfolio(initial_money=10_000, share_holdings={
    "Apple": 0
})

n = len(stock_data)

value_of_portfolio = [np.nan] * n

for i in range(n):
    price = stock_data.Close.iloc[i]
    if i <= 50 and i % 10 == 0:
        portfolio.buy(ticker="Apple", shares=2, price=price)
        pass
    if stock_data["SMA 50 D"].iloc[i] > 0:
        if stock_data["SMA 10 D"].iloc[i] > 0:
            portfolio.buy(ticker="Apple", shares=5, price=price)
            print(f"Bought! Price of Share {price}, Shares: {portfolio.portfolio['Apple']}, Money = {portfolio.money}")
        else:
            portfolio.add("Apple", 0)
    
    if stock_data["SMA 50 D"].iloc[i] <= 0:
        if stock_data["SMA 10 D"].iloc[i] > 0:
            portfolio.add("Apple", 0)
        else:
            portfolio.sell(ticker="Apple", shares=5, price=price)              
            print(f"Sold! Shares: {portfolio.portfolio['Apple']}, Money = {portfolio.money}")
    value_of_portfolio[i] = (portfolio.get_value(ticker="Apple", price=price))

    # this lags too much so i make way less money

Bought! Price of Share 261.6300048828125, Shares: 17, Money = 5960.829986572266
Bought! Price of Share 258.0199890136719, Shares: 22, Money = 4670.730041503906
Bought! Price of Share 249.02000427246094, Shares: 27, Money = 3425.6300201416016
Bought! Price of Share 250.0800018310547, Shares: 32, Money = 2175.230010986328
Bought! Price of Share 260.4800109863281, Shares: 37, Money = 872.8299560546875
Bought! Price of Share 269.19000244140625, Shares: 37, Money = 872.8299560546875
Bought! Price of Share 262.510009765625, Shares: 37, Money = 872.8299560546875
Bought! Price of Share 259.5199890136719, Shares: 37, Money = 872.8299560546875
Bought! Price of Share 257.54998779296875, Shares: 37, Money = 872.8299560546875
Bought! Price of Share 249.85000610351562, Shares: 37, Money = 872.8299560546875
Bought! Price of Share 248.97999572753906, Shares: 37, Money = 872.8299560546875
Bought! Price of Share 242.83999633789062, Shares: 37, Money = 872.8299560546875
Bought! Price of Share 251.4400024

In [30]:
# trade when price is above sma10

portfolio = Portfolio(initial_money=10_000, share_holdings={
    "Apple": 0
})

n = len(stock_data)

value_of_portfolio = [np.nan] * n

for i in range(n):
    price = stock_data.Close.iloc[i]
    if price > stock_data["SMA 10"].iloc[i]:
        portfolio.buy(ticker="Apple", shares=1, price=price)
    elif price < stock_data["SMA 10"].iloc[i]:
        portfolio.sell(ticker="Apple", shares=1, price=price)
    else:
        portfolio.add("Apple", 0)
    value_of_portfolio[i] = portfolio.get_value("Apple", price=price)

In [31]:
portfolio_fig.add_trace(
    go.Scatter(
        x = stock_data.index,
        y = value_of_portfolio
    )
)

portfolio_fig.show()

In [18]:
# trade when sma10 is above sma10

portfolio = Portfolio(initial_money=10_000, share_holdings={
    "Apple": 0
})

n = len(stock_data)

value_of_portfolio = [np.nan] * n

for i in range(n):
    max_shares = int(portfolio.money // price)
    price = stock_data.Close.iloc[i]
    if stock_data["SMA 10"].iloc[i] > stock_data["SMA 50"].iloc[i]:
        portfolio.buy(ticker="Apple", shares=max_shares, price=price)
    elif stock_data["SMA 10"].iloc[i] < stock_data["SMA 50"].iloc[i]:
        portfolio.sell(ticker="Apple", shares=5, price=price)
    else:
        portfolio.add("Apple", 0)
    value_of_portfolio[i] = portfolio.get_value("Apple", price=price)

In [19]:
portfolio_fig.add_trace(
    go.Scatter(
        x = stock_data.index,
        y = value_of_portfolio
    )
)

portfolio_fig.show()